Import Libraries





In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle"

In [5]:
!cd /content/drive/MyDrive/

In [6]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

 99% 226M/228M [00:08<00:00, 31.8MB/s]
100% 228M/228M [00:08<00:00, 28.7MB/s]


In [7]:
!mkdir dataset

In [8]:
!unzip the-movies-dataset.zip -d dataset

Archive:  the-movies-dataset.zip
  inflating: dataset/credits.csv     
  inflating: dataset/keywords.csv    
  inflating: dataset/links.csv       
  inflating: dataset/links_small.csv  
  inflating: dataset/movies_metadata.csv  
  inflating: dataset/ratings.csv     
  inflating: dataset/ratings_small.csv  


#### **Content Based**

In [ ]:
df_movie= pd.read_csv('/content/dataset/movies_metadata.csv')
df_movie.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
df_movie.shape

(45466, 24)

In [ ]:
df_credit= pd.read_csv('/content/dataset/credits.csv')
df_credit.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [ ]:
df_credit.shape

(45476, 3)

In [ ]:
df_movie.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
df_credit.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [ ]:
df_movie['id'].dtype

dtype('O')

In [ ]:
df_credit.dtypes

cast    object
crew    object
id       int64
dtype: object

In [ ]:
df_movie = df_movie.drop([19730, 29503, 35587]) # Causes value error while converting
df_movie['id'] = df_movie['id'].astype('int')

In [ ]:
df = df_movie.merge(df_credit, on='id')
df.shape

(45538, 26)

#### Let's select important columns:)

In [ ]:
df = df[['genres', 'id', 'overview','release_date', 'title', 'vote_average', 'vote_count', 'cast', 'crew']]

In [ ]:
df.dtypes

genres           object
id                int64
overview         object
release_date     object
title            object
vote_average    float64
vote_count      float64
cast             object
crew             object
dtype: object

In [ ]:
# Some conversions
df['id'] = df['id'].astype('int32')
df['vote_average'] = df['vote_average'].astype('float32')
df['vote_count'] = df['vote_count'].astype('float32')

In [ ]:
df['cast'] = df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
df.head()

,genres,id,overview,release_date,title,vote_average,vote_count,cast,crew
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story,7.7,5415.0,[],"[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji,6.9,2413.0,[],"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men,6.5,92.0,[],"[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale,6.1,34.0,[],"[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,"[{'id': 35, 'name': 'Comedy'}]",11862,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II,5.7,173.0,[],"[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45538 entries, 0 to 45537
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        45538 non-null  object 
 1   id            45538 non-null  int32  
 2   overview      44584 non-null  object 
 3   release_date  45451 non-null  object 
 4   title         45535 non-null  object 
 5   vote_average  45535 non-null  float32
 6   vote_count    45535 non-null  float32
 7   cast          45538 non-null  object 
 8   crew          45538 non-null  object 
dtypes: float32(2), int32(1), object(6)
memory usage: 3.0+ MB


#### Based on overview

In [ ]:
df['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [ ]:
df['overview'] = df['overview'].fillna('')
df['overview'].isnull().sum()

0

#### Get a sample from data cause the data is too large

In [ ]:
sampled_data = df.sample(n=30000, random_state=42)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# initialize vectorizer
# use tfidf to minimize the importance of the words that oftern appear
# in the whole corpus
vectorizer= TfidfVectorizer(
    tokenizer= word_tokenize,
    token_pattern= None,
    stop_words= 'english'
)

# fit and transform
overview_transformed= vectorizer.fit_transform(sampled_data['overview'])

# check output shape
overview_transformed.shape

(30000, 71466)

There are around 71k different words used by 30k movies' overview.

In [ ]:
overview_transformed = overview_transformed.astype(np.float32)

In [ ]:
# calculate cosine similarities
overview_similarities= cosine_similarity(
    overview_transformed, overview_transformed
)

# turn into DF for us to see
overview_similarities_df= pd.DataFrame(overview_similarities,index= sampled_data['title'], columns= sampled_data['title'])
overview_similarities_df.iloc[:5, :5]

title,Teen Witch,Precious,Broken Blade: Book Three - The Mark of the Assassin's Dagger,The Concubine,The Lost Continent
title,,,,,
Teen Witch,1.000000,0.016785,0.011977,0.007848,0.025365
Precious,0.016785,1.000000,0.035249,0.040323,0.017025
Broken Blade: Book Three - The Mark of the Assassin's Dagger,0.011977,0.035249,1.000000,0.105334,0.011852
The Concubine,0.007848,0.040323,0.105334,1.000000,0.053233
The Lost Continent,0.025365,0.017025,0.011852,0.053233,1.000000


#### Recommender function


In [ ]:
def get_recommendations(target_movie: str):

    # get all movie titles
    movie_titles= sampled_data['title'].values

    # check if the target movie is recognized
    if target_movie not in movie_titles:
        # movie is not recognized, print error message and exit the function
        print(f'Movie {target_movie} not recognized.\n')
        return

    # movie is recognized, proceed with recommendation logic

    similarities = overview_similarities
    # get the index number of target movie
    index_of_target_movie= np.where(movie_titles == target_movie)[0]

    # get similarity of all movies to target movie
    similarity_scores= similarities[index_of_target_movie].ravel()

    # make a dictionary of {title: similarity_score, title: similarity_score, ...}
    titles_and_scores= dict(zip(movie_titles, similarity_scores))

    # sort dictionary by score
    # get top 10 movies
    top_10_most_similar= sorted(titles_and_scores.items(), key= lambda x: x[1], reverse= True)[1:11]

    # print the result
    print(f'Movie recommendations similar to {target_movie}:')
    for i, (movie, rating) in enumerate(top_10_most_similar):
        print(f'\t{i+1}. {movie}')
    print('\n')

In [ ]:
 get_recommendations('Up')

Movie recommendations similar to Up:
	1. The time and patience
	2. I Love You Phillip Morris
	3. The Butterfly
	4. New Waterford Girl
	5. Possible Loves
	6. Mr. Destiny
	7. Rabbit Without Ears 2
	8. Raising Helen
	9. Committed
	10. Beautiful Ohio




In [ ]:
get_recommendations('Sherlock Holmes')

Movie recommendations similar to Sherlock Holmes:
	1. The Sign of Four: Sherlock Holmes' Greatest Case
	2. Without a Clue
	3. Young Sherlock Holmes
	4. The Seven-Per-Cent Solution
	5. Sherlock Holmes and the Leading Lady
	6. The Adventures of Sherlock Holmes and Doctor Watson: The Treasures of Agra
	7. Murder by Decree
	8. The Case of the Whitechapel Vampire
	9. Приключения Шерлока Холмса и доктора Ватсона: Двадцатый век начинается
	10. H.H. Holmes: America's First Serial Killer




In [ ]:
get_recommendations('Toy Story')

Movie recommendations similar to Toy Story:
	1. Toy Story 3
	2. Toy Story 2
	3. The 40 Year Old Virgin
	4. Small Fry
	5. Andy Kaufman Plays Carnegie Hall
	6. Hot Splash
	7. Andy Peters: Exclamation Mark Question Point
	8. Superstar: The Life and Times of Andy Warhol
	9. Life Begins for Andy Hardy
	10. Welcome to Happiness




#### **Collaborative Filtering**

In [9]:
!pip install scikit-surprise #Surprise is a Python SciKit that comes with various recommender algorithms and similarity metrics to make it easy to build and analyze recommenders.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3096314 sha256=c23d3637df5f5b415bbc3731886bc484cbdb6ebbe5a6c585145597dca41d53f1
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [10]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.prediction_algorithms.knns import KNNBasic

ratings = pd.read_csv("/content/drive/MyDrive/kaggle/ratings_small.csv")
movie = pd.read_csv("/content/drive/MyDrive/kaggle/movies_metadata.csv")

ratings.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [11]:
# movie dataframe with votes more than 50
movie_md = movie[movie['vote_count']>50][['id','title']]

# IDs of movies with count more than 50
movie_ids = [int(x) for x in movie_md['id'].values]

# Select ratings of movies with more than 50 counts
ratings = ratings[ratings['movieId'].isin(movie_ids)]

# Reset Index
ratings.reset_index(inplace=True, drop=True)

# Print first 5 rows
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1371,2.5,1260759135
1,1,2105,4.0,1260759139
2,1,2294,2.0,1260759108
3,2,17,5.0,835355681
4,2,62,3.0,835355749


In [22]:
# Initialize a surprise reader object
# The Reader class is used to parse a file containing ratings
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0,5), skip_lines=1)

# Load the data
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader=reader)

# Matrix factorization
One of the popular algorithms to factorize a matrix is the singular value decomposition (SVD) algorithm

In [13]:
from surprise.model_selection import cross_validate
svd = SVD()
cross_validate(svd, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8930  0.8945  0.9109  0.8963  0.8858  0.8961  0.0082  
MAE (testset)     0.6877  0.6882  0.7049  0.6942  0.6812  0.6913  0.0080  
Fit time          1.10    1.65    1.34    0.74    0.77    1.12    0.35    
Test time         0.06    0.21    0.27    0.28    0.19    0.20    0.08    


{'test_rmse': array([0.89295943, 0.89453538, 0.910939  , 0.89632154, 0.88578234]),
 'test_mae': array([0.6876916 , 0.68821693, 0.70493253, 0.69423846, 0.68123477]),
 'fit_time': (1.0971410274505615,
  1.652644157409668,
  1.342897653579712,
  0.7395391464233398,
  0.7725710868835449),
 'test_time': (0.062126874923706055,
  0.20965242385864258,
  0.2675790786743164,
  0.2789154052734375,
  0.18827605247497559)}

**Recommender function**

In [14]:
def get_recommendations(data, movie_md, user_id, top_n, algo):

    # creating an empty list to store the recommended product ids
    recommendations = []

    # creating an user item interactions matrix
    user_movie_interactions_matrix = data.pivot(index='userId', columns='movieId', values='rating')

    # extracting those product ids which the user_id has not interacted yet
    non_interacted_movies = user_movie_interactions_matrix.loc[user_id][user_movie_interactions_matrix.loc[user_id].isnull()].index.tolist()

    # looping through each of the product ids which user_id has not interacted yet
    for item_id in non_interacted_movies:

        # predicting the ratings for those non interacted product ids by this user
        est = algo.predict(user_id, item_id).est

          # appending the predicted ratings
        movie_name = movie_md[movie_md['id']==str(item_id)]['title'].values[0]
        recommendations.append((movie_name, est))

    # sorting the predicted ratings in descending order
    recommendations.sort(key=lambda x: x[1], reverse=True)

    return recommendations[:top_n] # returing top n highest predicted rating products for this user

In [15]:
get_recommendations(data=ratings, movie_md=movie_md, user_id=5, top_n=20, algo=svd)

[('Men in Black II', 4.819283683433931),
 ('While You Were Sleeping', 4.619937045204484),
 ('Gremlins 2: The New Batch', 4.595805172765068),
 ('License to Wed', 4.594537803154102),
 ('Shriek If You Know What I Did Last Friday the Thirteenth',
  4.560962710559578),
 ('Galaxy Quest', 4.509632015668412),
 ('Space Jam', 4.508440568132529),
 ('Beverly Hills Cop III', 4.4920031239464935),
 ('Terminator 3: Rise of the Machines', 4.490318192621336),
 ('Young Frankenstein', 4.478856045983677),
 ('Lonely Hearts', 4.474545210469059),
 ('Beetlejuice', 4.474251058042146),
 ('Point Break', 4.4737372229287535),
 ('Sicko', 4.462311057706034),
 ('The Million Dollar Hotel', 4.461198928096542),
 ('The Thomas Crown Affair', 4.458387035756721),
 ('Birdman of Alcatraz', 4.454215990617364),
 ('The First Wives Club', 4.453857874306589),
 ('Hard Target', 4.450129702373883),
 ('The Dreamers', 4.448962155378586)]

#### **User Based Recommemder System**
For a user U, with a set of similar users determined based on rating vectors consisting of given item ratings, the rating for an item I, which hasn’t been rated, is found by picking out N users from the similarity list who have rated the item I and calculating the rating based on these N ratings.

**k-NN**

For the memory-based approaches discussed above, the algorithm that would fit the bill is Centered k-NN because the algorithm is very close to the centered cosine similarity formula.

In [16]:
#Declaring the similarity options.
sim_options = {'name': 'cosine',
               'user_based': True}

# KNN algorithm is used to find similar items
sim_user = KNNBasic(sim_options=sim_options, verbose=False, random_state=50)

# Train the algorithm on the trainset, and predict ratings for the testset
# sim_user.fit(trainset)
cross_validate(sim_user, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9579  0.9719  0.9681  0.9727  0.9611  0.9663  0.0059  
MAE (testset)     0.7430  0.7513  0.7483  0.7575  0.7496  0.7500  0.0047  
Fit time          0.06    0.10    0.05    0.06    0.09    0.07    0.02    
Test time         0.89    1.39    0.51    0.82    0.93    0.91    0.28    


{'test_rmse': array([0.95787289, 0.97187669, 0.96812257, 0.97273672, 0.96107079]),
 'test_mae': array([0.74299917, 0.75127556, 0.7483346 , 0.75752525, 0.74961657]),
 'fit_time': (0.05835270881652832,
  0.10352849960327148,
  0.052596330642700195,
  0.0617063045501709,
  0.08620047569274902),
 'test_time': (0.8944096565246582,
  1.3918206691741943,
  0.5126795768737793,
  0.8168771266937256,
  0.9267206192016602)}

In [17]:
get_recommendations(ratings, movie_md, 8,10,sim_user)

[('Rio Bravo', 5),
 ('The Celebration', 5),
 ('Gentlemen Prefer Blondes', 5),
 ('The Evil Dead', 5),
 ('Strangers on a Train', 5),
 ('The General', 5),
 ('Once Upon a Time in Mexico', 5),
 ('Blood: The Last Vampire', 5),
 ('Captain America: The First Avenger', 5),
 ('You, Me and Dupree', 5)]

#### **Item Based Recommemder System**
For an item I, with a set of similar items determined based on rating vectors consisting of received user ratings, the rating by a user U, who hasn’t rated it, is found by picking out N items from the similarity list that have been rated by U and calculating the rating based on these N ratings.

In [18]:
#Declaring the similarity options.
sim_options = {'name': 'cosine',
               'user_based': False}

# KNN algorithm is used to find similar items
sim_item = KNNBasic(sim_options=sim_options, verbose=False, random_state=50)

# Train the algorithm on the trainset, and predict ratings for the testset
#sim_item.fit(trainset)
cross_validate(sim_item, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9817  0.9831  0.9771  0.9815  0.9785  0.9804  0.0022  
MAE (testset)     0.7684  0.7695  0.7572  0.7637  0.7596  0.7637  0.0048  
Fit time          0.26    0.20    0.17    0.11    0.11    0.17    0.06    
Test time         1.17    0.77    0.58    0.59    0.58    0.74    0.23    


{'test_rmse': array([0.98171633, 0.98311376, 0.97713394, 0.98152744, 0.97845869]),
 'test_mae': array([0.76840093, 0.7694868 , 0.75718934, 0.7636945 , 0.75962118]),
 'fit_time': (0.26352858543395996,
  0.20397686958312988,
  0.17017245292663574,
  0.10937333106994629,
  0.10836958885192871),
 'test_time': (1.1661713123321533,
  0.7740552425384521,
  0.583873987197876,
  0.5945107936859131,
  0.5782957077026367)}

In [19]:
get_recommendations(ratings, movie_md, 8,10,sim_item)

[('Sliver', 5),
 ('Vanilla Sky', 5),
 ('Ratatouille', 5),
 ('Fireworks', 5),
 ('Fanaa', 5),
 ('Babysitter Wanted', 5),
 ('Wild Strawberries', 4.75),
 ('Wild Things', 4.75),
 ('Live Free or Die Hard', 4.75),
 ('You, Me and Dupree', 4.75)]